In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Новый раздел

# Новый раздел

In [ ]:
ls

drive/  sample_data/


In [ ]:
# импортируем необходимые библиотеки
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing, model_selection, metrics
from sklearn.model_selection import train_test_split

import lightgbm as lgb

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/train.csv.zip", parse_dates=["activation_date"])
data.head()

FileNotFoundError: ignored

In [ ]:
# Вы можете сделать EDA, например, посмотреть deal probability по категориям. Если вам интересно =)

In [ ]:
  train, test = train_test_split(data, random_state=13, test_size=0.3)
  val, test = train_test_split(test, random_state=13, test_size=0.3)

In [ ]:
# Давайте обучим простенькую модель

# Отделите целевую переменную
y_train = train['deal_probability']
y_val = val['deal_probability']
y_test = test['deal_probability']

# Давайте добавим фактор: день недели активации
train["activation_weekday"] = train['activation_date'].dt.dayofweek
val["activation_weekday"] = val['activation_date'].dt.dayofweek
test["activation_weekday"] = test['activation_date'].dt.dayofweek

# Давайте заэнкодим категориальные переменные
cat_vars = ["region", "city", "parent_category_name", "category_name", "user_type", "param_1", "param_2", "param_3"]
for col in cat_vars:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train[col].values.astype('str')) + list(val[col].values.astype('str')) + list(test[col].values.astype('str')))
    train[col] = lbl.transform(list(train[col].values.astype('str')))
    val[col] = lbl.transform(list(val[col].values.astype('str')))
    test[col] = lbl.transform(list(test[col].values.astype('str')))

cols_to_drop = ["item_id", "user_id", "title", "description", "activation_date", "image", "deal_probability"]
X_train = train.drop(cols_to_drop, axis=1)
X_val = val.drop(cols_to_drop, axis=1)
X_test = test.drop(cols_to_drop, axis=1)

In [ ]:
X_val.head()

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,item_seq_number,user_type,image_top_1,activation_weekday
128604,15,1153,4,29,126,203,143,550.0,8,1,377.0,0
839577,5,329,4,29,207,193,133,2500.0,66,1,46.0,1
230058,10,743,5,15,284,188,0,13000.0,61,1,2218.0,1
153103,16,1276,6,0,278,100,665,3470000.0,1,1,1118.0,2
1063751,16,1063,4,29,126,122,149,5500.0,69,0,536.0,3


In [ ]:
params = {
    "objective" : "regression",
    "metric" : "rmse",
    "num_leaves" : 30,
    "learning_rate" : 0.1,
    "bagging_fraction" : 0.7,
    "feature_fraction" : 0.7,
    "bagging_frequency" : 5,
    "bagging_seed" : 2018,
    "verbosity" : -1
}

lgtrain = lgb.Dataset(X_train, label=y_train) # Создать lgb Dataset с тренировочной выборкой
lgval =  lgb.Dataset(X_val, label=y_val) # Создать lgb Dataset с валидационной выборкой
model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=20)

pred_test_y = model.predict(X_test, num_iteration=model.best_iteration)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: bagging_frequency
Training until validation scores don't improve for 100 rounds
[20]	valid_0's rmse: 0.236582
[40]	valid_0's rmse: 0.233712
[60]	valid_0's rmse: 0.232541
[80]	valid_0's rmse: 0.231955
[100]	valid_0's rmse: 0.231508
[120]	valid_0's rmse: 0.231172
[140]	valid_0's rmse: 0.230845
[160]	valid_0's rmse: 0.230558
[180]	valid_0's rmse: 0.230358
[200]	valid_0's rmse: 0.230217
[220]	valid_0's rmse: 0.230054
[240]	valid_0's rmse: 0.229902
[260]	valid_0's rmse: 0.229783
[280]	valid_0's rmse: 0.229633
[300]	valid_0's rmse: 0.229514
[320]	valid_0's rmse: 0.229363
[340]	valid_0's rmse: 0.229257
[360]	valid_0's rmse: 0.229162
[380]	valid_0's rmse: 0.229071
[400]	valid_0's rmse: 0.229002
[420]	valid_0's rmse: 0.228929
[440]	valid_0's rmse: 0.228867
[460]	valid_0's rmse: 0.228819
[480]	valid_0's rmse: 0.228769
[500]	valid_0's rmse: 0.228719
[520]	valid_0's rmse: 0.228663
[540]	valid_0's rmse: 0.228621
[560]	valid_0's rmse: 0.228583
[580]	valid_0's 

In [ ]:
print(metrics.mean_squared_error(y_test, pred_test_y))

0.05178713838770174
